# Create a PISM domain from a polygon based on the BedMachine v5 grid

Calibration of flow parameters by sector.

In [ ]:
from pathlib import Path
import geopandas as gp
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from pyDOE2 import lhs
from scipy.stats.distributions import randint, uniform

from pism_tutorials.domain import get_bounds, create_domain
from pism_tutorials.plotting import register_colormaps
from pism_tutorials.utils import merge_dicts, dict2str, sort_dict_by_key

xr.set_options(keep_attrs=True)

register_colormaps()

In [ ]:
# The name of the PISM Cloud S3 bucket
bucket_name = "pism-cloud-data"
! aws s3 sync --no-sign-request s3://pism-cloud-data/tutorial_files  tutorial_files

In [ ]:
# Coordinate Reference System 
crs = "EPSG:3413"

# the base resolution of BedMachine in meters
base_resolution: int = 150

# the resolutions that you want supported:
# 150, 300, 450, 600, 900, 1200, 1500, 1800, 2400, 3000, 3600, and 4500m
multipliers = [1, 2, 3, 4, 6, 8, 10, 12, 16, 20, 24, 30]

# buffer in m
buffer = 3e3

# Path to BedMachine file, open using xarray
ds = xr.open_dataset("tutorial_files/BedMachineGreenland-v5.nc")
ds = ds.rio.set_spatial_dims(x_dim="x", y_dim="y")
ds.rio.write_crs(crs, inplace=True)

In [ ]:
# Path to polygon file, open using GeoPandas
basins = gp.read_file("tutorial_files/Greenland_Basins_PS_v1.4.2_w_shelves.gpkg").to_crs(crs).dropna(subset=["SUBREGION1"])

In [ ]:
# Basic example:
# We loop over all basins in the "basins" geopandas dataframe assuming that an "Name" attribute exists.
# First we add the buffer, then we extract the bounding box, and finally we calculate the domain and 
# save it as a netCDF file.

# This should be parallelized with Dask in the future.

for m_id, basin in basins.iterrows():
    name = basin["SUBREGION1"]
    print(f"Processing basin {name}")
    minx, miny, maxx, maxy = basin.geometry.buffer(buffer).bounds
    ll = ds.sel({"x": minx, "y": miny}, method="nearest")
    ur = ds.sel({"x": maxx, "y": maxy}, method="nearest")
    tmp_ds = ds.sel({"x": slice(ll["x"], ur["x"]), "y": slice(ur["y"], ll["y"])})
    x_bnds, y_bnds = get_bounds(tmp_ds, 
                                base_resolution=base_resolution,
                                multipliers=multipliers)
    sub_ds = ds.sel({"x": slice(*x_bnds), "y": slice(*y_bnds[::-1])})
    grid = create_domain(x_bnds, y_bnds)
    grid.attrs.update({"domain": name})
    grid.to_netcdf(f"{name}_domain.nc", engine="h5netcdf")
    # Save a PDF
    fig = plt.figure()
    ax = fig.add_subplot(111)
    sub_ds["bed"].plot(ax=ax, cmap="bath_topo")
    basins.iloc[[m_id]].plot(ax=ax, alpha=0.5)
    ax.set_title(name)
    fig.savefig(f"{name}.png", dpi=600)

In [ ]:
n_samples = 3

flow_params_prior = {
            "basal_resistance.pseudo_plastic.q": uniform(0.25, 0.75),
            "basal_yield_stress.mohr_coulomb.till_effective_fraction_overburden": uniform(
                loc=0.01, scale=0.03
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_max": uniform(
                loc=40.0, scale=20.0
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_min": uniform(
                loc=5.0, scale=30.0
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_min": uniform(
                loc=-1000, scale=1000
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_max": uniform(
                loc=0, scale=1500
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_min": uniform(loc=5.0, scale=30.0),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_max": uniform(loc=40.0, scale=20.0),  
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_min": uniform(loc=-1000, scale=1000), 
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_max": uniform(loc=0, scale=1500), 
            "stress_balance.sia.enhancement_factor": uniform(loc=1.0, scale=3.0),
            "stress_balance.ssa.Glen_exponent": uniform(loc=2.75, scale=0.75),
            "stress_balance.sia.Glen_exponent": uniform(loc=1.0, scale=3.0),
        }

keys_prior = list(flow_params_prior.keys())
print("Prior Keys")
print("-" * 80)
print("\n".join([k for k in keys_prior]))


unif_sample = lhs(len(keys_prior), n_samples)
dist_sample = np.zeros_like(unif_sample)
for i, key in enumerate(keys_prior):
    dist_sample[:, i] = flow_params_prior[key].ppf(unif_sample[:, i])

df = pd.DataFrame(dist_sample, columns=flow_params_prior.keys())

master_config_file = "tutorial_files/pism_config.nc"

def check_params(d: dict) -> None:
    print("\nChecking parameters")
    print("------------------------------------------------------------")
    with xr.open_dataset(master_config_file) as m_ds:
        for key in d:
            if hasattr(m_ds["pism_config"], key) is False:
                print(f"  - {key} not found in pism_config")
    print("------------------------------------------------------------\n")


In [ ]:
resolution = "4500m"  # a coarse resolution such that we can run the simulation on a desktop computer

output_dir = Path("example_2")
output_dir.mkdir(parents=True, exist_ok=True)

extra_file = ""
start = "2015-01-01"
end = "2016-01-01"

input_params = {
    "bootstrap": "",
    "regional": "",
#    "i": "tutorial_files/BedMachineGreenland-v5.nc",
    "i": "tutorial_files/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc",
    "input.regrid.file": "tutorial_files/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc",
    "input.regrid.vars": "litho_temp,enthalpy,age,tillwat,bmelt,ice_area_specific_volume,thk"
}

grid_params = {
    "grid.dx": resolution,
    "grid.dy": resolution,
    "grid.Mz": 101,
    "grid.Lz": 4000,
    "grid.Mbz": 11,
    "grid.Lbz": 1000,
    "grid.file": None
}

time_params = {
    "time.start": start,
    "time.end": end,
    "time.calendar": "standard",
    "time_stepping.skip.enabled": "",
    "time_stepping.skip.max": 100
}

climate_params = {
    "surface.models": "given,forcing",
    "surface.given.file": "tutorial_files/MARv3.9_ERAI_climate_1978-2018_MEAN.nc",
#    "surface.force_to_thickness.file": "tutorial_files/BedMachineGreenland-v5.nc",
    "surface.force_to_thickness.file": "tutorial_files/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc",
}

stress_balance = {
    "stress_balance": "ssa+sia",
    "stress_balance.calving_front_stress_bc": "",  
    "stress_balance.ice_free_thickness_standard": 5,
    "stress_balance.sia.bed_smoother.range": resolution[:-1], 
    "stress_balance.sia.enhancement_factor": 2.608046,
    "stress_balance.sia.flow_law": "gpbld",
    "stress_balance.ssa.Glen_exponent": 3.309718,
    "stress_balance.ssa.enhancement_factor": 1.0,
    "stress_balance.sia.max_diffusivity": 100000,
    "basal_resistance.pseudo_plastic.enabled": "yes",
    "basal_resistance.pseudo_plastic.q": 0.7508221,
    "basal_yield_stress.mohr_coulomb.till_effective_fraction_overburden": 0.01845403,
    "basal_yield_stress.mohr_coulomb.topg_to_phi.enabled": "yes",
    "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_max": 42.79528,
    "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_min": 7.193718, 
    "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_max": 243.8239, 
    "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_min": -369.6359, 
}

output_params = {
    "output.file": None,  # do not write a state file
    "output.format": "netcdf4_parallel",
    "output.extra.file": None,
    "output.extra.times": "yearly",
    "output.extra.vars": "velsurf_mag,usurf,thk,climatic_mass_balance,ice_surface_temp,mask,mass_fluxes,ice_mass_transport_across_grounding_line,ice_mass",
}

run_dict = merge_dicts(input_params, grid_params, time_params, stress_balance,climate_params, output_params)

In [ ]:
n = 8
for m_id, basin in basins.iterrows():
    name = basin["SUBREGION1"]
    for s_id, sample in df.iterrows():
        run_dict.update(sample.to_dict())
        state_file = Path(output_dir) / f"state_g{resolution}_basin_{name}_id_{s_id}.nc"
        spatial_file = Path(output_dir) / f"spatial_g{resolution}_basin_{name}_id_{s_id}.nc"
        run_dict.update({"grid.file": f"{name}_domain.nc", "output.file": state_file, "output.extra.file": spatial_file})
        run_str = dict2str(sort_dict_by_key(run_dict))
        cmd = f"mpirun -np {n} pism " + run_str
        ! $cmd